In [9]:
import pandas as pd
import re

adqs = pd.read_csv('adqs_2017_06_01.csv')
df = adqs[~pd.isnull(adqs['ADQ Value'])]


In [11]:
for bnf_code in df[df['BNF Name'].str.contains('\(\d+ ?D')]['BNF Code'].unique():
    sql = """
    SELECT
      quantity,
      COUNT(*) AS count
    FROM
      tmp_eu.raw_prescribing_data_2018_04
    WHERE
      BNF_Code = '{}'
    GROUP BY
      quantity
    ORDER BY
      count DESC
    """
    dose_quantity = int(re.search('\((\d+) ?D', str(df[df['BNF Code'] == bnf_code]['BNF Name'])).group(1))
    quantities = pd.io.gbq.read_gbq(sql.format(bnf_code), 'ebmdatalab', dialect='standard',verbose=False)
    if quantities.empty:
        continue
    dose_quantity_match = quantities[quantities.quantity == dose_quantity]
    if not dose_quantity_match.empty:
        dose_quantity_count = dose_quantity_match.iloc[0]['count']
    else:
        dose_quantity_count = 0
    print("Code {}: most frequent quantity {} has count {}; count of q matching dose ({}): {}".format(
        bnf_code, 
        quantities.iloc[0]['quantity'], 
        quantities.iloc[0]['count'], 
        dose_quantity, 
        dose_quantity_count))

Code 0105020B0AAAAAA: most frequent quantity 1 has count 91; count of q matching dose (14): 1
Code 0105020B0BBAAAA: most frequent quantity 1 has count 197; count of q matching dose (14): 1
Code 0105020D0AAACAC: most frequent quantity 1 has count 763; count of q matching dose (14): 1
Code 0105020D0BCAAAC: most frequent quantity 1 has count 2; count of q matching dose (14): 0
Code 0301011E0AAABAB: most frequent quantity 1 has count 311; count of q matching dose (60): 0
Code 0301011E0AAACAC: most frequent quantity 1 has count 711; count of q matching dose (60): 0
Code 0301011E0AAADAD: most frequent quantity 1 has count 872; count of q matching dose (100): 0
Code 0301011E0AAAEAE: most frequent quantity 1 has count 191; count of q matching dose (120): 0
Code 0301011E0BCAAAB: most frequent quantity 1 has count 274; count of q matching dose (60): 0
Code 0301011E0BCABAC: most frequent quantity 1 has count 709; count of q matching dose (60): 0
Code 0301011E0BDAAAD: most frequent quantity 1 has 

KeyboardInterrupt: 